In [ ]:
"""
Format: term_i, df_i:
    <doc1, tf_1: <pos1, pos2, … >; 
    doc2, tf_2: <pos1, pos2, …>; …>

Format: to, 993427: 
    <1, 6: <7, 18, 33, 72, 86, 231>;   
    2, 5: <1, 17, 74, 222, 255>; … >
"""

In [13]:
import re
import math
import nltk

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from collections import Counter
from tqdm.auto import tqdm

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# initialize nltk tokenizer
nltk.download('punkt')
sent_segmenter = nltk.data.load('tokenizers/punkt/english.pickle')

word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()

stemmer = nltk.stem.porter.PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wirl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wirl\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wirl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
inverted_index = eval(open('all_docs_freq_all_tokens_dict.txt','r').read())

In [3]:
inverted_index['usernam, 2']

[{'13185,1': [0]}, {'13191,8': [841, 848, 850, 948, 955, 969, 980, 982]}]

In [10]:
# ref: https://mathpretty.com/10661.html
term = "usernam"

NUM_OF_ALL_DOCS = 13173 # number of all documents (processed)

def calculate_weight(raw_count_in_doc, term_doc_in_all_document):
    """
    raw_count_in_doc : appear times of term in the doc (raw count)
    term_doc_in_all_document : number of docs which contains the term (processed)
    """
    idf = NUM_OF_ALL_DOCS / term_doc_in_all_document
    w_i_j = (1 + math.log10(raw_count_in_doc)) * math.log10(idf)

    return w_i_j

def cosine_similarity(vec_a, vec_b):
    assert len(vec_a) == len(vec_b), "Vectors needs to be same length"

    dot = sum(i * j for i, j in zip(vec_a, vec_b))
    a_norm = sum(i**2 for i in vec_a) ** 0.5
    b_norm = sum(i**2 for i in vec_b) ** 0.5

    cos_sim = dot / (a_norm * b_norm)

    return cos_sim

In [51]:
term_doc_in_all_document = 0

keys = ['usernam, 2', 'member, 3'] # for test
# keys = ['usernam, 2'] # for test
d_j = []
q = []
term_weight_doc_id_dict = dict()
# for term_df_key in list(inverted_index.keys()):
for term_df_key in list(keys):
    d_j = []
    term = re.findall('\w+', term_df_key)[0] # 'usernam, 2' -> 'usernam'
    term_doc_in_all_document = int(re.findall(', ([0-9]+)', term_df_key)[0]) # 'usernam, 2' -> 2

    # print(term_df_key)
    # print(inverted_index[term_df_key])
    
    temp_dict = dict()
    for i in range(len(inverted_index[term_df_key])):
        doc_raw_count_and_posting = list(inverted_index[term_df_key][i].keys())[0] # [{'13185,1': [0]}, {'13191,8': ...}...]
        print(doc_raw_count_and_posting)
        doc_id = re.findall('([0-9]+),', doc_raw_count_and_posting)[0] # '13185,1' -> 13185 #doc的編號
        raw_count_in_doc = int(re.findall(',([0-9]+)', doc_raw_count_and_posting)[0]) # '13185,1' -> 1 #在這個doc出現幾次
        weight_i_j = calculate_weight(raw_count_in_doc, term_doc_in_all_document)
        temp_dict[doc_id]=weight_i_j
    

    term_weight_doc_id_dict[term] = temp_dict

print(term_weight_doc_id_dict)
    # print(term, doc_id, d_j)
    # term_weight_doc_id_dict = {'usernam':{'13185': 8.79, '13191': 27.10}, 'password':{'13191': 30.52}, 'becom': {'13194': 16.73}}


usernam 13191 [8.792777378669882, 27.076843926618768]
member 13193 [30.52191019395961, 8.387312270561717, 8.387312270561717]


In [41]:
queries = ('user name', 'passwordsss', 'became')
terms = ('usernam', 'password', 'becom')

corrected_queries_by_terms = []

term_weight_doc_id_dict = {'usernam':{'13185': 8.79, '13191': 27.10}, 'password':{'13191': 30.52}, 'becom': {'13194': 16.73}}

# d_1, d_2, ..., d_j, {'13185': (8.79, 0.00), ...}
term_weight_for_tokenized_queries = {}
q = []

all_docs_ids_for_tokenized_queries = set()

for query in queries:
    query = stemmer.stem(query) # stemmize the query token

    if query in term_weight_doc_id_dict:
        term_docs_weights = term_weight_doc_id_dict[query]
        all_docs_ids_for_tokenized_queries = all_docs_ids_for_tokenized_queries.union(set(term_docs_weights.keys())) # get all query tokens doc id

        for doc_id in all_docs_ids_for_tokenized_queries:
            term_weight_for_tokenized_queries.update({doc_id: []}) # create dict with all involved doc ids, like {'13191': [], '13185': []}
    else:
        # get most similar terms for user's query
        search_range = [t for t in terms if t[0] == query[0]] # optimize: reduce the search range by starts with same character

        # find most similar for "correct" query tokens to terms by using Edit distance / Levenshtein distance
        for i in range(len(terms)):
            if i == 0:
                edit_distance = nltk.edit_distance(query, terms[i])
                corrected_queries_by_terms.append(terms[i])

            tmp = nltk.edit_distance(query, terms[i])

            if tmp < edit_distance:
                edit_distance = tmp

                if corrected_queries_by_terms:
                    corrected_queries_by_terms.pop()

                corrected_queries_by_terms.append(terms[i]) # append most similar token into corrected_queries

print(f"corrected_queries_by_terms: {corrected_queries_by_terms}")
print(f"all_docs_ids_for_tokenized_queries: {all_docs_ids_for_tokenized_queries}")

# TODO: 把這個修好，原本可以輸出 {'13185': [8.79, 0.0, 0.0], '13191': [27.10, 30.52, 0.0], '13194': [0.0, 0.0, 16.73]} 的但 correct 完 queries 就不行了
for corrected_query in corrected_queries_by_terms:
    if corrected_query in term_weight_doc_id_dict:
        term_docs_weights = term_weight_doc_id_dict[corrected_query] # {'13191': 30.52}

        for doc_id, weight in term_docs_weights.items(): # 13185, 13191
            doc_id = str(doc_id)
            term_weight_for_tokenized_queries[doc_id].append(weight)

        need_to_0_padding = set(all_docs_ids_for_tokenized_queries) - set(term_docs_weights.keys())

        if need_to_0_padding:
            for k in need_to_0_padding:
                term_weight_for_tokenized_queries[k].append(0.0)

all_docs_ids_for_tokenized_queries
term_weight_for_tokenized_queries

corrected_queries_by_terms: ['usernam', 'password', 'becom']
all_docs_ids_for_tokenized_queries: set()


KeyError: '13185'

In [ ]:
# cosine similarity
# term = "usernam"
# query = "username"

# query -> stemmize -> search key (use cosine similarity to get most similiar key) -> get weight

# d_j = query 在 doc 的 tf-idf weight 向量
# q = query 各字在 query 的 tf-idf weight 向量

# Hong Kong Business
# d_j w_ij 就是第 i 個 term 在第 j 個 doc 的 weight

# d_j 有很多，有 13173 個，每個長度都跟 query 一樣
# d_j = (Hong在j個doc的weight, kong在j個doc的weight, business在j個doc的weight)
# q = (Hong在query內的weight, kong在query內的weight, business在query內的weight)